In [1]:
import json
import numpy as np
import pyvista as pv

from scipy.spatial import cKDTree
from sklearn.cluster import DBSCAN
from scipy.spatial.transform import Rotation

# Set path of main directory
from pathlib import Path
mainDir = Path(".").parent.absolute().parent.absolute()

# Helper Functions

## Reading Icosphere Files

Our Icospheres are saved as NPZ files and were originally generated by the *stripy* library. Since *stripy* itself is a bit difficult to install, I would rather avoid having it required to run my code.

## Spherical Polar Coordinates

Just like most of our notebooks so far, it is useful to transform between cartesian and polar coordinates. Our spherical coordinate system is [based on this article](https://mathworld.wolfram.com/SphericalCoordinates.html), in particular we use equations 1 to 6 of their article.


In [2]:
# =========================================== Reading Icosphere Files =================================================
# Read NPZ files for Icosphere data
def getIcosphere(subdivisions=4):
    fileDir = str(mainDir) + '/Data/SpheresNPZ/IcosphereSubs{}.npz'.format(subdivisions)
    data = np.load(fileDir)
    return data['vertices'], data['cells']

# Converts cells format to pyvista faces for plotting
def cellsToFaces(cells):
    faces = []
    for c in cells:
        faces.append(3)
        for x in c:
            faces.append(x)
    return np.array(faces)

# ================================================ Spherical Coordinate Transformations =============================
# Coordinate transformation function between polar and cartesian
def polarToCartesian(radius, theta, phi, useLonLat=True):
    if useLonLat == True:
        theta, phi = np.radians(theta+180.), np.radians(90. - phi)
    X = radius * np.cos(theta) * np.sin(phi)
    Y = radius * np.sin(theta) * np.sin(phi)
    Z = radius * np.cos(phi)
    
    #Return data either as a list of XYZ coordinates or as a single XYZ coordinate
    if (type(X) == np.ndarray):
        return np.stack((X, Y, Z), axis=1)
    else:
        return np.array([X, Y, Z])

# Coordinate transformation function between polar and cartesian
def cartesianToPolarCoords(XYZ, useLonLat=True):
    X, Y, Z = XYZ[:, 0], XYZ[:, 1], XYZ[:, 2]
    R = (X**2 + Y**2 + Z**2)**0.5
    theta = np.arctan2(Y, X)
    phi = np.arccos(Z / R)
    
    #Return results either in spherical polar or leave it in radians
    if useLonLat == True:
        theta, phi = np.degrees(theta), np.degrees(phi)
        lon, lat = theta - 180, 90 - phi
        lon[lon < -180] = lon[lon < -180] + 360
        return R, lon, lat
    else:
        return R, theta, phi

# Gaussian Particles on a Sphere

We create a class of Guassian particles on an icosphere that initiates a flat terrain. A gaussian particle is created for each point on the icosphere, each with equal mass, and the mass is set based on a desired height of the terrain. In the class bellow, we have two Icospheres each with their own subdivisions, the first is used purely for initiating the location of particles, and the other is used for plotting.

## Radius of Sphere

The influence that various forces will have on our particles depends on the average distance that particles are away from each other. Therefore we set the radius of the sphere such that the average distance between particles is constant. This will help significantly in getting consistent simulations regardless of the number of particles that are involved. The area of a sphere is $A = 4 \pi r^2$, and the desired area of the sphere is equal to the number of particles $A = N$, so the desired radius of the sphere is: 

$$r = \sqrt{\frac{N}{4 \pi}}$$ 

After setting this as our radius of our sphere, all particles should be approximately 1 unit distance away from it's nearest neighbours. This radius is only necessary for simplifying our simulations with gaussian particles, but we can always change the radius after evaluating the final landscape.

## RBF Evaluations

In our previous notebooks, when evaluating the particles by summing the RBFs, we included all distances between every pair of particles. Due to memory concerns, this is infeasible when we have too many particles. Instead we only sum the RBFs at a predefined number of neighbouring particles. 

Since the vertices in these icosphere are not trully equally spaced appart, our resulting initial "flat" landscape will not trully be flat. This is not really an issue, because our landscape will be distorted significantly later once we begin simulating moving tectonic plates.

## Realistic Heights

Our gaussian particles are not capable of representing negative elevations, but instead we can define a *maxOceanDepth* and shift the elevations accordingly.

According to a random person on the internet, the average height of earths topology is 1800 meters bellow sea level (we can always change this number later). Therefore, we make sure that the average height of a flat terrain is about -1800 meters.

The maximum height that mountains may grow towards depends on how dense particles might move towards each other, and is therefore much harder for us to control. This height will be influnced by most forces that are involved in the simulation.

In [3]:
# Gaussian radial basis function
def gaussRBF(radius, std):
    coeff = 1/(std*(2*np.pi)**0.5)
    return coeff * np.exp(-0.5*(radius / std)**2)

class GaussParticlesSphere:
    def __init__(self, subdivisions=5, smoothness=2, plotSubdivisions=5, numNeighbs=100,
                    maxOceanDepth = -6000, maxMountainHeight=8000, averageTerrainHeights=-1800):
        
        # Set up particles from icosphere
        self.subdivisions = subdivisions
        self.XYZ, _ = getIcosphere(subdivisions=subdivisions)
        self.std = smoothness

        # Set radius of sphere
        self.numParticles = self.XYZ.shape[0]
        self.radius = (self.numParticles / (4 * np.pi))**0.5
        self.XYZ *= self.radius

        # Set number of neighbours used in distance calculations
        if numNeighbs > self.numParticles:
            numNeighbs = self.numParticles
        self.numNeighbs = numNeighbs

        # Set up sphere for plotting
        self.plotSubdivisions = plotSubdivisions
        self.plotXYZ, self.plotCells = getIcosphere(subdivisions=plotSubdivisions)
        self.plotFaces = cellsToFaces(self.plotCells)
        self.plotXYZ *= self.radius
        self.mesh = pv.PolyData(self.plotXYZ, self.plotFaces)
        self.plotKDTree = cKDTree(self.plotXYZ)

        # Height parameters
        self.maxOceanDepth = maxOceanDepth
        self.averageTerrainHeights = averageTerrainHeights
        self.maxMountainHeight = maxMountainHeight # Not used yet

        # Set mass of particles to a constant such that we get our desired average terrain height
        self.w = 1
        self.w = (averageTerrainHeights - maxOceanDepth) / np.mean(self.evaluate(shiftZero=False))
    
    # Evaluate the resulting landscape of particles using RBF
    # This function is optimized but only works for points on our icosphere for plotting
    def evaluate(self, shiftZero=True):
        dist, idx = self.plotKDTree.query(self.XYZ, k=self.numNeighbs)
        RBF = self.w * gaussRBF(dist, self.std)
        heights = np.zeros(self.plotXYZ.shape[0])
        np.add.at(heights, idx, RBF)
        if shiftZero:
            heights += self.maxOceanDepth
        return heights

    # Evaluate the resulting landscape of particles using RBF
    # This function is slower, but can evaluate any point in space
    def evaluateFromAnyPoints(self, XYZEval, numNeighbs=100):
        dist, _ = cKDTree(self.XYZ).query(XYZEval, k=numNeighbs)
        RBF = self.w * gaussRBF(dist, self.std)
        heights = np.sum(RBF, axis=1)
        return heights + self.maxOceanDepth
    
    # Glyphs are for visualizing particle locations
    def getGlyphs(self, size=0.2, scalars=None):
        glyphGeom = pv.Sphere(theta_resolution=1, phi_resolution=1)
        pointCloud = pv.PolyData(self.XYZ)
        if isinstance(scalars, np.ndarray):
            pointCloud['color'] = scalars
        glyphs = pointCloud.glyph(geom=glyphGeom, factor=size, scale=False, indices='color')
        return glyphs
    
    # These calculations may be used by various algorithms,
    # So we calculate them once for each iteration rather than each time they are used in code
    def iterationPrecompuations(self):
        self.kDTree = cKDTree(self.XYZ)

# Evaluate Gaussian Particles
parts = GaussParticlesSphere(subdivisions=5, smoothness=2, plotSubdivisions=5, numNeighbs=100)

# Plot icosphere
plotter = pv.PlotterITK()
plotter.add_mesh(parts.mesh, scalars=parts.evaluate())
plotter.add_mesh(parts.getGlyphs())
plotter.background_color = 'white'
plotter.show()


Viewer(background=(1.0, 1.0, 1.0), geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints',…

Viewer(background=(1.0, 1.0, 1.0), geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints',…

# Particle Simulator

We will be using the same framework as the 1D case to run our tectonic simulations. The *ParticleSimulator* bellow takes care of all time related parameters, and provides a flexible framework for specifying which forces to simulate. As inputs, the particle simulator will take time related parameters, particles, and a list of force objects. A force object in our context is any class that will manipulate particles at every iteration of the simulation. It may not necessarily be a force according to the phsyical definition of a force.

As of now, the list of forces is empty, and so our simulation should do nothing until we define some forces later on. To make a class compatible with our *ParticleSimulator*, it must have a *\__call__()* function that takes a *ParticleSimulator* as input. The class can then modify the particle object that has been attached to the simulator.

In [4]:
class ParticleSimulator:
    def __init__(self, particles, startTime=10, endTime=0, dt=1, forces=[]):

        # Time related parameters
        self.dt = dt
        self.endTime = endTime
        self.iterations = int((startTime - endTime)//dt) + 1
        self.time = np.linspace(startTime, endTime, self.iterations)

         # Particles related parameters
        self.particles = particles

        # Forces to be included in this simulation
        self.forces = forces
    
    # This gets called every simulation iteration
    def simulationIteration(self):
        self.particles.iterationPrecompuations()
        for force in self.forces:
            force(self)
    
    # Run the simulation
    def runSimulation(self, printCurrentIterations=False):
        for i, t in enumerate(self.time):
            if printCurrentIterations:
                print('Current Simulations Time: {} MYA'.format(t))
            self.currentTime = t
            self.simulationIteration()


# Test that the simulator does not throw any errors (it should do nothing at this stage)
parts = GaussParticlesSphere(subdivisions=4, smoothness=2, plotSubdivisions=4, numNeighbs=1000)
sim = ParticleSimulator(parts)
sim.runSimulation()


# Moving Tectonic Plates

In the code bellow, we provide a very simple example force *MoveParticles* to demonstrate how to use our simulation framework. We won't use it in our final code, but as a minimal example for demonstration. At the very least, a force object needs to have a *\__call__()* function with a particle simulator as an input argument. Any optional parameters should be set in the *\__init__()* function.

Similarly to our previous notebook, we can move particles by simply updating their XYZ locations. However, to ensure they remain on the surface of the sphere, we will be moving them using rotational quaternions. To create an appropriate rotational quaternion, we specify the axis that the particles should rotate about, and an angle of rotation 

In [5]:
# A simple force compatible with our simulation framework
class MoveParticles:

    # Set force parameters here
    def __init__(self, speed=0.02, axis=[0, 0, 1]):
        self.speed = speed
        self.axis = axis

    # This function is required by all force type objects
    def __call__(self, sim):
        XYZ = sim.particles.XYZ
        quat = quaternion(self.axis, self.speed * np.pi)
        rot = Rotation.from_quat(quat)
        pointsToRotate = (XYZ[:, 0] > 0)
        XYZ[pointsToRotate] = rot.apply(XYZ[pointsToRotate])
        sim.particles.XYZ = XYZ

# Given an axis of rotation and angle, create a quaternion
def quaternion(axis, angle):
    return [np.sin(angle/2) * axis[0], 
            np.sin(angle/2) * axis[1], 
            np.sin(angle/2) * axis[2], 
            np.cos(angle/2)]

# Evaluate Gaussian Particles
parts = GaussParticlesSphere(subdivisions=4, smoothness=2, plotSubdivisions=4, numNeighbs=200)
forces = [
    MoveParticles(),
]

# Test that the simulator does not throw any errors (it should do nothing at this stage)
sim = ParticleSimulator(parts, startTime=5, endTime=0, dt=1, forces=forces)
sim.runSimulation()

# Plot icosphere
plotter = pv.PlotterITK()
plotter.add_mesh(sim.particles.mesh, scalars=sim.particles.evaluate())
plotter.add_mesh(sim.particles.getGlyphs())
plotter.background_color = 'white'
plotter.show()

Viewer(background=(1.0, 1.0, 1.0), geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints',…

Viewer(background=(1.0, 1.0, 1.0), geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints',…

## Identifying the Plates

Plate IDs are used for identifying which tectonic plate a particle. Using code in the *PyGplates* notebook, we can generate NPZ files containing the plate IDs for various subdivision icosphere, at various time steps. Once we have created those NPZ files, we will assign each gaussian particle a plate ID based on it's nearest point on the *PlateIdentifyiers* icosphere, where the nearest point is identified using a KDTree.

Note that the *PlateIdentifier* has its own icosphere, and its subdivision will specify how accurate it is at assigning particles to plate Ids. Having a too high subdivision will significantly slow down the run time of our simulation.

In [6]:
# Class for identifying which tectonic plate a particular point bellongs to
class PlateIdentifier:
    def __init__(self, subdivision=4, plateIdsDirFormat=None):

        # Set parameters
        self.subdivision = subdivision
        if type(plateIdsDirFormat) == type(None):
            self.plateIdsDirFormat = str(mainDir) + '/Data/PlateIDs/Sub{}/Time{}.npz'
        else:
            self.plateIdsDirFormat = plateIdsDirFormat
        
        # Prepare KDTree for identifying nearest neighbours
        verts, _ = getIcosphere(subdivisions=subdivision)
        self.kdTree = cKDTree(verts)
    
    # Identify plate IDs using nearest negihbours
    def identifyPoints(self, time, XYZ):
        plateIds = np.load(self.plateIdsDirFormat.format(self.subdivision, int(time)))['ids']
        _, idx = self.kdTree.query(XYZ)
        return plateIds[idx]

# Create icosphere to identify points from
verts, cells = getIcosphere(subdivisions=6)
faces = cellsToFaces(cells)
mesh = pv.PolyData(verts, faces)

# Identify plate IDs
time = 20
PI = PlateIdentifier(subdivision=4)
plateIds = PI.identifyPoints(time, verts)

# Plot results
plotter = pv.PlotterITK()
plotter.add_mesh(mesh, scalars=plateIds)
plotter.background_color = 'white'
plotter.show()


Viewer(background=(1.0, 1.0, 1.0), geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints',…

Viewer(background=(1.0, 1.0, 1.0), geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints',…

# Plate Rotations

We create a class for moving tectonic plates using rotations. This class will be called by the simulator class as one of it's "forces". This class will inherit all functions and properties of the *PlateIdentifier* class. The two classes are kept seperately here to make the notebook neater, but they could just as easily be a single class instead, which I might do in the final version of the code.

This class assigns plate ids to particles, and then moves them using rotational quaternions. The rotations are saved as a JSON file, which was created in the *PyGplates* notebook.

The functions *convertList(), convertDict(), convertValue()* are taken from [StackOverflow User *thebjorn*](https://stackoverflow.com/questions/41843604/how-can-i-convert-nested-dictionary-keys-to-strings) and then modified for our own needs. All it does is help read our JSON file by traversing a nested dictionary from a JSON file and converts the data types of dictionary keys and values using recursion.

In [7]:
class PlateMover(PlateIdentifier):
    def __init__(self, dataDir=str(mainDir)+'/Data/PlateRotations.json', subdivision=4, plateIdsDirFormat=None):

        # Call the parent's initiator function to inherit all its properties
        super().__init__(subdivision=subdivision, plateIdsDirFormat=plateIdsDirFormat)

        # Load JSON containing plate rotations
        self.dataDir = dataDir
        with open(self.dataDir, 'r') as f:
            self.rotationsData = json.load(f)
        self.rotationsData = self.convertDict(self.rotationsData)
    
    # This function will be called by the simulator to move tectonic plates by one iteration
    def __call__(self, sim):

        # Get simulation parameters
        dt = sim.dt
        time = sim.currentTime
        XYZ = np.copy(sim.particles.XYZ)

        # Set up dictionaries of rotations
        plateIds = self.identifyPoints(time, XYZ)
        axis = self.rotationsData[time]['axii']
        angle = self.rotationsData[time]['angles']

        # Apply rotations to each plate
        for idx in np.unique(plateIds):
            quat = quaternion(axis[idx], dt * angle[idx])
            rot = Rotation.from_quat(quat) 
            XYZ[plateIds==idx] = rot.apply(XYZ[plateIds==idx])

        # Apply changes to coordinates
        sim.particles.XYZ = XYZ
    
    # Convert JSON data to prefered data types
    def convertDict(self, d):
        dic = {}
        for k, v in d.items():
            if str.isnumeric(k):
                dic[int(k)] = self.convertValue(v)
            else:
                dic[k] = self.convertValue(v)
        return dic

    # Used for processing lists during recursion
    def convertList(self, lst):
        return [self.convertValue(item) for item in lst]

    # Use recursion to traverse through nested dictionary
    def convertValue(self, v):
        if isinstance(v, dict):
            return self.convertDict(v)
        elif isinstance(v, list):
            return self.convertList(v)
        else:
            return v


# Initiate particles and forces
parts = GaussParticlesSphere(subdivisions=4, smoothness=2, plotSubdivisions=4, numNeighbs=100)
forces = [
    PlateMover(subdivision=4)
]

# Run simulation
sim = ParticleSimulator(parts, forces=forces, startTime=20, endTime=0, dt=1)
sim.runSimulation()

# Plot results
plotter = pv.PlotterITK()
plotter.add_mesh(parts.mesh, scalars=parts.evaluate())
plotter.add_mesh(parts.getGlyphs(size=0.5))
plotter.background_color = 'white'
plotter.show()

Viewer(background=(1.0, 1.0, 1.0), geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints',…

Viewer(background=(1.0, 1.0, 1.0), geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints',…

# Particle Animator

To better see how our particles move over time, we create a class for animations. Note that as of writting, evaluating the particles at each frame seriously slows down the performance of our code, especially when the number of neighbours used for evaluations is large. When debugging our code, it is significantly faster to simply plot the particle locations (Glyphs) without evaluating the final surface. As such, I have create a function for animating the resulting surface (slow), and a function for animating the location of particles with glyphs. The resulting animation will be saved as an MP4 file in the same directory of this notebook.

In [8]:
class ParticleAnimator(ParticleSimulator):
    def __init__(self, particles, startTime=10, endTime=0, dt=1, forces=None, lookAtLonLat=[0, 0], cameraZoom=1.4):
        
        # Call the parent's initiator
        super().__init__(particles, startTime=startTime, endTime=endTime, dt=dt, forces=forces)
        self.lookAtLonLat = lookAtLonLat
        self.cameraZoom = cameraZoom

    # Creates an animate mp4 movie of our simulation
    # This runs significantly slower, because the particles need to be evaluated at each iteration
    def animate(self, animationDir='Animation.mp4', framesPerIteration=4, clim=[-6000, 10000]):

        # Set up plotter for animation
        plotter = pv.Plotter(notebook=False, off_screen=True)
        plotter.add_mesh(self.particles.mesh, scalars=self.particles.evaluate(), clim=clim)

        # Set up camera positions
        plotter.camera_position = 'yz'
        plotter.camera.zoom(self.cameraZoom)
        plotter.camera.elevation = self.lookAtLonLat[0]
        plotter.camera.azimuth = 180 + self.lookAtLonLat[1]

        #Write initial frames of movie
        plotter.open_movie(animationDir)
        for i in range(framesPerIteration):
            plotter.write_frame()

        # Run simulation and animate
        for i, t in enumerate(self.time):
            self.currentTime = t
            self.simulationIteration()

            # Write animation frame
            plotter.update_scalars(self.particles.evaluate())
            for i in range(framesPerIteration):
                plotter.write_frame()
        plotter.close()
    
    # This animation will run faster, but only shows the particle position 
    # and does not show the final landscape evaluations
    def animateGlyphs(self, glyphSize=0.5, animationDir='Animation.mp4', framesPerIteration=4):

        # Set up plotter for animation
        plotter = pv.Plotter(notebook=False, off_screen=True)
        plotter.add_mesh(self.particles.getGlyphs(size=glyphSize))

        # Set up camera positions
        plotter.camera_position = 'yz'
        plotter.camera.zoom(self.cameraZoom)
        plotter.camera.elevation = self.lookAtLonLat[0]
        plotter.camera.azimuth = 180 + self.lookAtLonLat[1]

        #Write initial frames of movie
        plotter.open_movie(animationDir)
        for i in range(framesPerIteration):
            plotter.write_frame()
        
        # Run simulation and animate
        for i, t in enumerate(self.time):
            self.currentTime = t
            self.simulationIteration()

            # Write animation frame
            plotter.update_coordinates(self.particles.getGlyphs(size=glyphSize).points)
            for i in range(framesPerIteration):
                plotter.write_frame()
        plotter.close()

# Initiate particles and forces
parts = GaussParticlesSphere(subdivisions=5, smoothness=2, plotSubdivisions=5, numNeighbs=100)
forces = [
    PlateMover(subdivision=4)
]

# Run simulation (uncomment appropriate line)
ani = ParticleAnimator(parts, forces=forces, startTime=100, endTime=0, dt=1, lookAtLonLat=[0, 0], cameraZoom=1.4)
#ani.animate()
#ani.animateGlyphs()

# Particle Collisions

As we can see in the above simulations, the particles begin to accumulate at converging boundaries. To limit the maximum height that mountains can grow towards, we need to set a minimum distance that particles can be away from each other, which can be done using particle collisions.

Let $r$ be the minimum radi that particles can be away from each other. We can use *scipy.spatial.cKDTree.query_pairs()* to find all pairs of colliding particles (all pairs who have a distance of less than $2r$). We then reposition the particles such that they are a distance $2r$ away from each other.

Let $\textbf{p}_1$ and $\textbf{p}_2$ be the position of two colliding particles. They are currently a distance $d = \sqrt{(\textbf{p}_1 - \textbf{p}_2)^2}$ away from each other. The total distance we need to move both particles is $2r - d$, so each particle needs to be moved a distance $r - \frac{d}{2}$. The direction that we move each particle is $\pm (\textbf{p}_1 - \textbf{p}_2)$. Thus the final position of each particle is 

$$\textbf{p}_{t+1} = \textbf{p}_t \pm (\textbf{p}_1 - \textbf{p}_2) (r - \frac{\sqrt{(\textbf{p}_1 - \textbf{p}_2)^2}}{2})$$

## Project Onto Sphere

The above particle collider might cause particles to move of from the surface of the sphere. The class *ProjectOntoSphere* takes any point and places them back onto the surface of a sphere of fixed radius.

In [9]:
# Prevents particles from moving too close to each other
class ParticleCollider:
    def __init__(self, particleRadius=0.2):
        self.radius = particleRadius
    
    def __call__(self, sim):
        XYZ = np.copy(sim.particles.XYZ)

        # Get indices of all pairs of colliding particles
        pairs = sim.particles.kDTree.query_pairs(2*self.radius, output_type='ndarray')
        if pairs.shape[0] == 0:
            return XYZ
        
        # Get coordinates of pairs
        XYZ1 = XYZ[pairs[:, 0]]
        XYZ2 = XYZ[pairs[:, 1]]

        # Calculate direction and distance that particle need to move
        # to avoid being too close to each other
        displacement = XYZ2 - XYZ1
        distance = np.linalg.norm(displacement, axis=1)
        direction = displacement / distance[:, None]
        distToMove = self.radius - (distance/2)

        # Move particles accordingly
        XYZ[pairs[:, 0]] = XYZ1 - direction * distToMove[:, None]
        XYZ[pairs[:, 1]] = XYZ2 + direction * distToMove[:, None]
        sim.particles.XYZ = XYZ

# Particles may be moved off from the surface of the sphere
# This class projects them back onto the surface of the sphere
class ProjectOntoSphere:
    def __call__(self, sim):
        r = sim.particles.radius
        XYZ = sim.particles.XYZ
        length = np.linalg.norm(XYZ, axis=1)
        sim.particles.XYZ = r * XYZ / length[:, None]

# Initiate particles and forces
parts = GaussParticlesSphere(subdivisions=6, smoothness=2, plotSubdivisions=6, numNeighbs=100)
forces = [
    PlateMover(subdivision=4),
    ParticleCollider(particleRadius=0.8),
    ProjectOntoSphere()
]

# Run simulation
sim = ParticleAnimator(parts, forces=forces, startTime=80, endTime=0, dt=1)
sim.runSimulation()

# Plot results
plotter = pv.PlotterITK()
plotter.add_mesh(parts.mesh, scalars=parts.evaluate())
plotter.add_mesh(parts.getGlyphs(size=0.5))
plotter.background_color = 'white'
plotter.show()

Viewer(background=(1.0, 1.0, 1.0), geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints',…

Viewer(background=(1.0, 1.0, 1.0), geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints',…

# Spawn and Despawn Particles

To represent volcanic activity and plate submergion, we will spawn and despawn particles from converging and diverging plate boundaries. To ensure that mass is always conserved, we ensure that the number of spawend particles equals the number of despawned particles. 

For now, we keep the spawning and despawning algorithms very simple, which we can improve later by taking into properties such as Mass, Density, Age, etc... into consideration. As of now, we despawn particles by identyfying clusters of particles with *sklearn.cluster.DBSCAN()* and removing clustered particles at random, which normally happens at converging plate boundaries. We spawn new particles at a random location of the sphere. The spawner first calculates how many particles are missing (despawned), and spawns sufficient particles to replace them.

Although we are already getting better than expected results, we can clearly improve this to make it more consistent with actual tectonic activity.

In [ ]:
class Despawner:
    def __init__(self, clusterSizeCoeff=1, maxClusterDistance=0.7, probDespawn=0.02):
        self.clusterSizeCoeff = clusterSizeCoeff
        self.maxClusterDistance = maxClusterDistance
        self.probDespawn = probDespawn # To Do: Adjust this based on dt

    def __call__(self, sim):
        XYZ = sim.particles.XYZ

        # Get clusters of particles
        numParticles = XYZ.shape[0]
        clusterSize = self.clusterSizeCoeff * np.log10(numParticles)
        clustering = DBSCAN(eps=self.maxClusterDistance, min_samples=int(clusterSize)).fit(XYZ)

        # Choose random particles withing cluster to despawn
        isInCluster = (clustering.labels_ != -1)
        inClusterIdx = np.argwhere(isInCluster)
        numInCluster = np.sum(isInCluster)
        randNum = np.random.random(numInCluster)
        probDespawn = self.probDespawn * sim.dt
        despawnThisParticle = (probDespawn > randNum)
        despawnIdx = inClusterIdx[despawnThisParticle]

        # Despawn the particles
        sim.particles.XYZ = np.delete(sim.particles.XYZ, despawnIdx[:, 0], 0)

class Spawner:
    def __call__(self, sim):
        XYZ = sim.particles.XYZ

        # Get number of missing particles
        currentNumParts = XYZ.shape[0]
        requiredNumParts = sim.particles.numParticles
        missingNumParts = requiredNumParts - currentNumParts

        # Spawn particles at random coordinates (for now)
        randCoords = 2 * np.random.random((missingNumParts, 3)) - 1
        sim.particles.XYZ = np.concatenate((XYZ, randCoords), axis=0)



# Initiate particles and forces
parts = GaussParticlesSphere(subdivisions=6, smoothness=2, plotSubdivisions=6, numNeighbs=100)
forces = [
    PlateMover(subdivision=4),
    ParticleCollider(particleRadius=0.8),
    Despawner(clusterSizeCoeff=1, maxClusterDistance=0.7, probDespawn=0.04),

    # The spawner places particles randomly, but not necessarily on the surface of the sphere
    Spawner(),

    # But by projecting them onto a sphere, they have now been moved to the surface of the sphere
    ProjectOntoSphere()
]

# Run simulation
sim = ParticleAnimator(parts, forces=forces, startTime=100, endTime=0, dt=1, lookAtLonLat=[20, -30], cameraZoom=1.4)
#sim.animate()
sim.runSimulation()

# Plot results
plotter = pv.PlotterITK()
plotter.add_mesh(parts.mesh, scalars=parts.evaluate())
plotter.background_color = 'white'
plotter.show()

In [ ]:
# Plot results
plotter = pv.PlotterITK()
plotter.add_mesh(parts.mesh, scalars=parts.evaluate())
plotter.background_color = 'white'
plotter.show()